In [30]:
import kaolin as kal
from kaolin.io.modelnet import ModelNet
from kaolin.ops.conversions import trianglemeshes_to_voxelgrids, voxelgrids_to_cubic_meshes
from kaolin.io.usd import *
from PIL import Image
import torch
import numpy as np
import meshplot as mp

In [60]:
data_dir = "data/ModelNet10"
output_dir = "output"
data = ModelNet(data_dir, categories=['toilet'])

print(set(data.labels))
print(data.get_data(10).vertices.shape)
print(data.get_data(10).faces.shape)

{'toilet'}
torch.Size([11348, 3])
torch.Size([10249, 3])


In [64]:
original = data.get_data(23).vertices.numpy(), data.get_data(23).faces.numpy()
d = mp.subplot(*original, s=[1, 2, 0])
verts = data.get_data(23).vertices.cuda().unsqueeze(dim=0)
voxel_grid = trianglemeshes_to_voxelgrids(verts, data.get_data(23).faces.cuda(), 20)
voxel_grid.cpu().numpy()np.save('toilet.npy')
c_v, c_f = voxelgrids_to_cubic_meshes(voxel_grid)
cube_mesh = c_v[0].cpu().numpy(), c_f[0].cpu().numpy()
mp.subplot(*cube_mesh, s=[1, 2, 1],data=d)

AttributeError: 'numpy.ndarray' object has no attribute 'save'

In [52]:
max_val = torch.max(verts, dim=1)[0]
print(max_val)
origin = torch.min(verts, dim=1)[0]
scale = torch.max(max_val - origin, dim=1)[0]
print(scale)

tensor([[15.8606, 34.6507, 12.5580]], device='cuda:0')
tensor([69.3014], device='cuda:0')


In [22]:
vertices = torch.tensor([[[0, 0, 0],
                          [1, 0, 0],
                          [0, 0, 1]]], dtype=torch.float)
faces = torch.tensor([[0, 1, 2]], dtype=torch.long)
origin = torch.zeros((1, 3))
scale = torch.ones((1))
voxel_grid = trianglemeshes_to_voxelgrids(vertices, faces, 3, origin, scale)
c_v, c_f = voxelgrids_to_cubic_meshes(voxel_grid)
cube_mesh = c_v[0].cpu().numpy(), c_f[0].cpu().numpy()
d = mp.subplot(vertices.numpy(), faces.numpy())
mp.plot(*cube_mesh, data=d)

ValueError: could not broadcast input array from shape (3,3) into shape (3,)

In [40]:
d = mp.subplot(*original, s=[1, 2, 0])
mp.subplot(*cube_mesh,s=[1, 2, 1], data=d)
